# CONUS404 Explorer Panel App
Using hvplot and Panel from the [holoviz](https://holoviz.org) suite

In [ ]:
import xarray as xr
import cf_xarray 
import metpy
import intake
import numpy as np
import hvplot.xarray
from geoviews import tile_sources as gvts
import panel as pn

In [ ]:
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'

In [ ]:
hytest_cat = intake.open_catalog(url)
list(hytest_cat)

In [ ]:
# open the conus404 sub-catalog
cat = hytest_cat['conus404-catalog']
list(cat)

In [ ]:
dataset = 'conus404-hourly-osn'

This app views the entire spatial domain, so tell Dasks to read in bigger chunks to speed up loading. The native chunk size in the Zarr dataset is `{'x':175, 'y':175} ` so pick multiples of that.  This reduces the time to load the entire domain on a 2 core machine on AWS us-west-2 from 15s to 5s. 

In [ ]:
cat[dataset].kwargs.update({'chunks':{'x':2*175, 'y':2*175}})

In [ ]:
ds = cat[dataset].to_dask()

Add `standard_name='time'` so `cf_xarray` can find the time coordinate

In [ ]:
ds.time.attrs['standard_name'] = 'time'

Find data variables that have a time dimension (but are not time `bounds` variables)

In [ ]:
time_vars = []
for var in ds.data_vars:
    if len(ds[var].dims) > 0:
        if 'time' in ds[var].dims[0] and not 'bounds' in var:
            time_vars.append(var)

In [ ]:
print(len(time_vars))

In [ ]:
init_var = 'T2'

In [ ]:
ds[init_var]

Use Metpy to parse the CRS

In [ ]:
ds = ds.metpy.parse_cf()

In [ ]:
crs = ds.T2.metpy.cartopy_crs

Create widget for variable selection

In [ ]:
var_select = pn.widgets.Select(name='CONUS404 Variables:', options=time_vars, value=init_var)

Create widget for basemap selection

In [ ]:
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)

The `plot` function below creates the `hvplot` panel layout object.   ROMS is on a C-Grid and variables have different coordinates depending on where they are on the grid.   So we use `cf-xarray` to determine the time, depth, longitude and latitude coordinates for each variable.  

We specify a basemap, specify the CRS for the selected variable, and indicate we want to `rasterize` the plot so that we can render massive meshes in the browser. 

We also specify the `groupby` parameter as the list of dimensions that remains after we remove Y and X: `ds[var].dims[:-2]`, which automatically handles variables with either dimensions [T, Y, X] or [T, Z, Y, X].  

We also specify which `bokeh` controls we want to be active by default:  the `wheel_zoom` and `pan` controls.

We also change the default slider to a selection widget for the time dimension (and vertical dimension if it exists) so that specific values are easy to select.  See https://stackoverflow.com/a/54912917/2005869

In [ ]:
time_vals = 48

One might think that it would be faster to make an image map using the projected coordinates instead of quadmesh with 2D lon,lat coordinates, but it seems the same.   And QuadMesh is a nicer looking map. 

In [ ]:
@pn.depends(var_select, base_map_select)
def plot(var, base_map):
    extra_dims = list(ds[var].dims[:-2])
    da = ds[var].cf.isel(T=slice(-time_vals,-1)).unify_chunks().load()

    mesh = da.hvplot.quadmesh(x=da.cf['longitude'].name, y=da.cf['latitude'].name,
                    rasterize=True, geo=True, title=var, attr_labels=False, 
                    cmap='turbo', grid=True, xlabel='Longitude', ylabel='Latitude',
                          frame_height=400).opts(alpha=0.7)

    return pn.panel(mesh * base_map, widgets={extra_dims[0]: pn.widgets.Select})

In [ ]:
col = pn.Column(base_map_select, var_select, plot)

We use `.servable()` below not only to display the panel object, but to make the panel servable outside the notebook via:

`panel serve CONUS404_Panel_App.ipynb`

In [ ]:
col.servable('CONUS404 Dashboard')